<a href="https://colab.research.google.com/github/mmsamiei/final-ms-thesis-code/blob/master/phase2/models/fatiha_hinge/fatiha_hinge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")


<IPython.core.display.Javascript object>

Done.


In [1]:
!nvidia-smi

Mon Aug 17 12:54:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
train_file = '/content/ drive/My Drive/Thesis/phase-2/history_sentence_pairs_train.csv'
valid_file =  '/content/drive/My Drive/Thesis/phase-2/history_sentence_pairs_valid.csv' 

In [5]:
!pip -q install transformers

     |████████████████████████████████| 778kB 4.5MB/s 
     |████████████████████████████████| 1.1MB 19.8MB/s 
     |████████████████████████████████| 3.0MB 31.7MB/s 
     |████████████████████████████████| 890kB 43.8MB/s 


In [6]:
from torch.utils.data import Dataset, DataLoader
import os
import torch
import json
from torch.utils.data.sampler import SubsetRandomSampler

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-2_H-128_A-2')

In [7]:
import random
import pandas as pd
import logging

class MyDataset(Dataset):
    """My dataset."""

    def __init__(self, csv_file, frac=1, split_rate=1, max_len=512):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
        """
        self.dialogues = pd.read_csv(csv_file)
        self.dialogues.dropna(inplace=True)
        
        self.dialogues = self.dialogues[self.dialogues.index % split_rate == 0]
        
        s = self.dialogues['history'].apply(len).sort_values().index
        self.dialogues = self.dialogues.reindex(s)
        s = self.dialogues['true_sentence'].apply(len).sort_values().index
        self.dialogues = self.dialogues.reindex(s)


        self.dialogues.dropna(inplace=True)

        self.max_len = max_len


    def __len__(self):
        return len(self.dialogues)

    @staticmethod
    def truncuate_join_pair_sentence(sentence1, sentence2, max_len=510):

        """
        truncuate sentence one from head and sentence two from tail
        Args:
            sentence1 (string): first sentence
            sentence2 (string): seconde sentence
        """
        temp1 = tokenizer.encode(sentence1,add_special_tokens=False)
        temp2 = tokenizer.encode(sentence2,add_special_tokens=False)
        ### two above line may cause warning but no problem because we've handle them below
        logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)
        seq_1 = temp1
        seq_2 = temp2
        num_tokens_to_remove = len(temp1) + len(temp2) + 3 - max_len
        if num_tokens_to_remove > 0 :
            seq_1, seq_2, _ = tokenizer.truncate_sequences(temp1[::-1],temp2, num_tokens_to_remove=num_tokens_to_remove)
            seq_1.reverse()
        result_list = [tokenizer.cls_token_id]+seq_1+[tokenizer.sep_token_id]+seq_2+[tokenizer.sep_token_id]
        token_type_ids = [0] * (len(seq_1) + 2) + [1] * (len(seq_2) + 1)
        return result_list, token_type_ids


    def __getitem__(self, idx):
      
        
        history = self.dialogues.iloc[idx].history
        true_sentence = self.dialogues.iloc[idx].true_sentence
        false_sentence = self.dialogues.iloc[idx].false_sentence


        true_pair, true_pair_segments = MyDataset.truncuate_join_pair_sentence(history, true_sentence, self.max_len)
        false_pair, false_pair_segments  = MyDataset.truncuate_join_pair_sentence(history, false_sentence, self.max_len)
        
        

        true_pair = torch.LongTensor(true_pair)
        false_pair = torch.LongTensor(false_pair)

        true_pair_segments = torch.LongTensor(true_pair_segments)
        false_pair_segments = torch.LongTensor(false_pair_segments)


        sample = {'true_pair': true_pair, 'false_pair': false_pair,
                  'true_pair_segments': true_pair_segments,
                  'false_pair_segments': false_pair_segments}

        return sample

In [8]:
train_dataset = MyDataset(train_file, split_rate=4, max_len=128)
valid_dataset = MyDataset(valid_file, split_rate=4, max_len=128)
print(len(train_dataset))
print(len(valid_dataset))

693921
36857


In [9]:
from tqdm.auto import tqdm

def my_collate_fn(batch):

  len_batch = len(batch)

  
  max_len_true_pair = max([len(data['true_pair']) for data in batch])
  max_len_false_pair = max([len(data['false_pair']) for data in batch])
  
  padding_ind = 0 ## for bert is 0
  result_true_pair = torch.zeros(len_batch, max_len_true_pair)
  result_false_pair = torch.zeros(len_batch, max_len_false_pair)
  
  result_true_segment = torch.zeros(len_batch, max_len_true_pair)
  result_false_segment = torch.zeros(len_batch, max_len_false_pair)

  for i, data in enumerate(batch):
    p1 = len(data['true_pair'])
    result_true_pair[i, :p1] = data['true_pair']
    p2 = len(data['false_pair'])
    result_false_pair[i, :p2] = data['false_pair']

    p3 = len(data['true_pair_segments'])
    result_true_segment[i, :p3] = data['true_pair_segments']

    p4 = len(data['false_pair_segments'])
    result_false_segment[i, :p4] = data['false_pair_segments']


  return result_true_pair.long(), result_false_pair.long(), result_true_segment.long(), result_false_segment.long()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
                                             shuffle=True, collate_fn=my_collate_fn,
                                           num_workers=1)

valid_sampler = torch.utils.data.SequentialSampler(valid_dataset)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=128, sampler=valid_sampler,
                                             shuffle=False, collate_fn=my_collate_fn, num_workers=1)


i = 0 
for batch_idx, batch  in tqdm(enumerate(train_loader)):
  true_batch, false_batch, a, p = batch
  print(false_batch.shape)
  i += 1 
  if(i==4):
    break

print(len(train_loader))
print(len(valid_loader))

torch.Size([128, 128])
torch.Size([128, 128])
torch.Size([128, 128])
torch.Size([128, 128])
5422
288


In [10]:
import os

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import AutoModel

class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()

    self.bert = AutoModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
    self.fc = nn.Linear(128,1)
    self.activation = nn.Tanh()
    nn.init.xavier_normal_(self.fc.weight)
  
  def forward(self, x, segments_tensors):
        temp = x
        temp = self.bert(temp, token_type_ids=segments_tensors)[0]
        ## temp = [batch, len, hid_size]
        temp = temp[:,0,:]
        temp = self.fc(temp)
        temp = self.activation(temp)
        temp = (temp - (-1))/2 
        return temp

dev = torch.device('cuda')
model = Model().to(dev)


x = torch.LongTensor(16, 40).random_(1,1000).to(dev)
y = torch.LongTensor(16, 40).random_(0,2).to(dev)
print(model(x, y).shape)


def count_parameters(model): return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))


torch.Size([16, 1])
4386049


In [11]:
from transformers import get_cosine_schedule_with_warmup, get_constant_schedule
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
scheduler = get_constant_schedule(optimizer)

In [12]:
import torch.nn.functional as F

def mahdi_loss(true_sml, false_sml):
  eps = 1e-8
  loss = torch.mean( -(true_sml)-(-false_sml))
  return loss

In [13]:
from tqdm.auto import tqdm

def train_step(batch_idx, batch):
  true_pairs, false_pairs, true_segments, false_segments = batch
  true_pairs = true_pairs.to(dev)
  false_pairs = false_pairs.to(dev)
  true_segments = true_segments.to(dev)
  false_segments = false_segments.to(dev)
  optimizer.zero_grad()
  true_sml = model(true_pairs, true_segments)
  false_sml = model(false_pairs, false_segments)
  loss = mahdi_loss(true_sml, false_sml)
  loss.backward()
  torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
  optimizer.step()
  scheduler.step()
  del true_pairs
  del false_pairs
  del true_segments
  del false_segments
  return loss.item()

def valid_step(batch_idx, batch):
  true_pairs, false_pairs, true_segments, false_segments = batch
  true_pairs = true_pairs.to(dev)
  false_pairs = false_pairs.to(dev)
  true_segments = true_segments.to(dev)
  false_segments = false_segments.to(dev)
  true_sml = model(true_pairs, true_segments)
  false_sml = model(false_pairs, false_segments)
  z = true_sml - false_sml
  num_err = z[z<0].size()[0]
  del true_pairs
  del false_pairs
  del true_segments
  del false_segments 
  return num_err

def valid_loop(valid_loader):
  total_error = 0
  model.eval()
  for batch_idx, batch in tqdm(enumerate(valid_loader),  total=len(valid_loader)):
    total_error += valid_step(batch_idx, batch)
  return total_error / len(valid_dataset)

In [14]:
new_learning = True
if new_learning:
  model_dir = "/content/drive/My Drive/Final-Thesis/Phase2/Fatiha_Plus/"
  step = 0
  log_list = []

In [ ]:
## if continue learning:
#!wget -q https://github.com/mmsamiei/MS-Thesis-Phase2/raw/master/Models/hashemi_16000steps.model
model_dir = "/content/drive/My Drive/Final-Thesis/Phase2/Fatiha"
checkpoint = torch.load(model_dir+'keshvari_25000steps.model')
step = checkpoint['log_list'][-1]['step']
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
optimizer._step = step
log_list = checkpoint['log_list']
new_learning = False
print(step)

25000


In [15]:
from tqdm.auto import tqdm

MAX_STEP = 6000
STEP_SAVE = 1000
STEP_CHECK = 5000
step_num = step + 1
log_list = log_list ### Check if new learning or not
print(step_num)
while step_num <= MAX_STEP:
  model.train()
  for batch_idx, batch in tqdm(enumerate(iter(train_loader)), total=len(train_loader)):
    step_loss = train_step(batch_idx, batch)
    log = {'step':step_num, 'train_loss':step_loss}

    if(step_num % STEP_CHECK == 0):
      valid_error = valid_loop(valid_loader)
      print("Error rate: {} at step {}".format(valid_error, step_num))  
      log['valid_error'] = valid_error
      log_list.append(log)
      model.train()
      torch.save({
            'model_state_dict': model.state_dict(),
            'log_list': log_list,
            'optimizer_state_dict': optimizer.state_dict() 
            }, model_dir+'fatiha_{}steps.model'.format(step_num))
      step_num += 1
      continue
    
    log_list.append(log)
    if(step_num % STEP_SAVE == 0):
      torch.save({
            'model_state_dict': model.state_dict(),
            'log_list': log_list,
            'optimizer_state_dict': optimizer.state_dict()
            }, model_dir+'fatiha_{}steps.model'.format(step_num))
    step_num += 1


1



Error rate: 0.1079849146702119 at step 5000


KeyboardInterrupt: ignored

In [ ]:
checkpoint = torch.load(model_dir+'motevaselian_5000steps.model')
step = checkpoint['log_list'][-1]['step']
model.load_state_dict(checkpoint['model_state_dict'])
model = model.eval()

In [ ]:
checkpoint['log_list'][-100:]

[{'step': 4901, 'train_loss': 0.965859055519104},
 {'step': 4902, 'train_loss': 0.9364367723464966},
 {'step': 4903, 'train_loss': 0.9242492318153381},
 {'step': 4904, 'train_loss': 0.9294930100440979},
 {'step': 4905, 'train_loss': 0.8620480298995972},
 {'step': 4906, 'train_loss': 0.805367648601532},
 {'step': 4907, 'train_loss': 1.0609407424926758},
 {'step': 4908, 'train_loss': 0.9978032112121582},
 {'step': 4909, 'train_loss': 0.9125474691390991},
 {'step': 4910, 'train_loss': 0.9051778316497803},
 {'step': 4911, 'train_loss': 0.8823820948600769},
 {'step': 4912, 'train_loss': 0.9469332695007324},
 {'step': 4913, 'train_loss': 0.8955069780349731},
 {'step': 4914, 'train_loss': 0.7936267852783203},
 {'step': 4915, 'train_loss': 1.1202847957611084},
 {'step': 4916, 'train_loss': 0.8871780633926392},
 {'step': 4917, 'train_loss': 0.8982921838760376},
 {'step': 4918, 'train_loss': 0.9952181577682495},
 {'step': 4919, 'train_loss': 0.9235968589782715},
 {'step': 4920, 'train_loss': 0.8

In [17]:
test_df = pd.read_csv('/content/drive/My Drive/Thesis/phase-2/history_sentence_pairs_test_unseen.csv')
test_df.fillna(' ', inplace=True)
mrr_df = pd.DataFrame(data=test_df.history.unique(), columns=['history'])
mrr_df['correct_rank'] = 0

In [18]:
test_df_gb = test_df.groupby(['history','true_sentence'])
for case in tqdm(test_df_gb):
  history = case[0][0]
  true_sentence = case[0][1]
  true_pair_untensor, true_segment_untensor = MyDataset.truncuate_join_pair_sentence(history, true_sentence)
  true_pair =  torch.LongTensor(true_pair_untensor).reshape(1, -1).to(dev)
  true_segment =  torch.LongTensor(true_segment_untensor).reshape(1, -1).to(dev)
  true_sml = model(true_pair, true_segment)
  case_df = case[1]
  for false_sentence in case_df['false_sentence']:
    false_pair_untensor, false_segment_untensor = MyDataset.truncuate_join_pair_sentence(history, false_sentence)
    false_pair =  torch.LongTensor(false_pair_untensor).reshape(1, -1).to(dev)
    false_segment =  torch.LongTensor(false_segment_untensor).reshape(1, -1).to(dev)
    false_sml = model(false_pair, false_segment)
    if(true_sml.item() < false_sml.item()):
      mrr_df.loc[mrr_df['history']==history, 'correct_rank'] += 1

In [25]:
len(mrr_df[ mrr_df['correct_rank'] < 1 ]) / len(mrr_df)

0.5446044983573414